## <img src="Imatges/IBM-DataScience.png" width="400" />
<h1><center>Capstone Project</center></h1>
<strong><center>Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto</center></strong>
<h3><center><strong>Part 2</strong></center></h3>
<h3><strong>Author:</strong> Jordi Planas </h3>
<h3>March 2020 </h3>

## Introduction
In this third part of the assignemnt of week 3, you will find, at the beginning, the two first parts which are necessary for uploading and preparing the basic geographical data on Toronto.
Once we have the geographical coordinates of each postal code, we will use the <strong>explore</strong> function to get the most common venue categories in each neighborhood in the city of Toronto. Then, we will use this information to cluster "neighborhoods" with the <strong>k</strong> means algorithm and show the results in a map.

In [1]:
import pandas as pd
#!pip install wikipedia
import wikipedia as wp


### Fetching Data

In [52]:
html = wp.page("List of postal codes of Canada: M").html().encode("UTF-8")
try: 
    df = pd.read_html(html)[0]  # Try 2nd table first as most pages contain contents table first
except:
    print("Something went wrong when reading the page")

### Data processing

In [53]:
indexNames = df[df['Borough'] == 'Not assigned' ].index
df.drop(indexNames, axis = 0, inplace = True)
df['Neighborhood'] = df['Neighborhood'].str.replace(' /',',')
df = df.reset_index()
df.drop('index', axis = 1, inplace = True)
df.rename(columns = {'Postal code' : 'Postal Code'}, inplace = True)

## Begining of part 2

In [4]:
#!pip install geocoder
import geocoder # import geocoder
import time

### Fetching the coordinates through "Geocoder"

In [9]:
# loop until you get the coordinates
start = time.time()

lat_lng_coords = None
i = 0
while(lat_lng_coords is None and i < 10 ):
    i += 1
    #print(i)
    g = geocoder.google('{}, Toronto, Ontario'.format("M3A"))
    lat_lng_coords = g.latlng
    #print(lat_lng_coords)

end = time.time()
if i == 10:
    print("the limit has been reached after : ", round(end - start, 2), " seconds")

the limit has been reached after :  3.11  seconds


After several attempts it was impossible to get even a single coordinate from a postal code. So, I switched to the geospatial data recorded in the .csv file

### Retrieving the coordinates through the downloaded csv file

In [58]:
import wget

url = 'http://cocl.us/Geospatial_data'
#wget.download(url, 'Dades/Geospatial_data.csv')
coordinates = pd.read_csv('Dades/Geospatial_data.csv')

### Merging the two dataframes to get a single one with all the relevant information

In [57]:
df = pd.merge(df, coordinates, how='left', on='Postal Code')
df.head(12)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
